Importing Required Libraries

In [1]:
from git import Repo
import json
import os
from pathlib import Path
import pandas as pd
import numpy as np
import psycopg2

Clone Repository

In [2]:
clone_path = r"D:\Labmentix\phonpe-pulse-visualization"
repo_url = "https://github.com/PhonePe/pulse.git"

if not os.path.exists(clone_path):
    os.makedirs(clone_path)

repo_name = os.path.basename(repo_url).removesuffix('.git').title()
repo_path = os.path.join(clone_path, repo_name)

# Clone only if the path doesn't exist
if not os.path.exists(repo_path):
    print(f"Cloning {repo_url} into {repo_path}...")
    Repo.clone_from(repo_url, repo_path)
else:
    print(f"Repository already exists at {repo_path}, skipping clone.")

directory = os.path.join(repo_path, 'data')
print(directory)

Repository already exists at D:\Labmentix\phonpe-pulse-visualization\Pulse, skipping clone.
D:\Labmentix\phonpe-pulse-visualization\Pulse\data


Renaming sub-directories and Extracting necessary paths

In [3]:
# Function to rename messy state names in a proper format

def rename(directory):
    for root, dirs, files in os.walk(directory):
        if 'state' in dirs:
            state_dir = os.path.join(root, 'state')
            for state_folder in os.listdir(state_dir):
                # rename the state folder
                old_path = os.path.join(state_dir, state_folder)
                new_path = os.path.join(state_dir, state_folder.title().replace('-', ' ').replace('&', 'and'))
                os.rename(old_path, new_path)
    print("Renamed all sub-directories successfully")
                
# Function to extract all paths that has sub-directory in the name of 'state'

def extract_paths(directory):
    path_list = []
    for root, dirs, files in os.walk(directory):
        if os.path.basename(root) == 'state':
            path_list.append(root.replace('\\', '/'))
    return path_list

In [4]:
rename(directory)

Renamed all sub-directories successfully


To check all directory path

In [5]:
state_directories = extract_paths(directory)
state_directories

['D:/Labmentix/phonpe-pulse-visualization/Pulse/data/aggregated/insurance/country/india/state',
 'D:/Labmentix/phonpe-pulse-visualization/Pulse/data/aggregated/transaction/country/india/state',
 'D:/Labmentix/phonpe-pulse-visualization/Pulse/data/aggregated/user/country/india/state',
 'D:/Labmentix/phonpe-pulse-visualization/Pulse/data/map/insurance/country/india/state',
 'D:/Labmentix/phonpe-pulse-visualization/Pulse/data/map/insurance/hover/country/india/state',
 'D:/Labmentix/phonpe-pulse-visualization/Pulse/data/map/transaction/hover/country/india/state',
 'D:/Labmentix/phonpe-pulse-visualization/Pulse/data/map/user/hover/country/india/state',
 'D:/Labmentix/phonpe-pulse-visualization/Pulse/data/top/insurance/country/india/state',
 'D:/Labmentix/phonpe-pulse-visualization/Pulse/data/top/transaction/country/india/state',
 'D:/Labmentix/phonpe-pulse-visualization/Pulse/data/top/user/country/india/state']

### Creating dataframes from cloned json files

##### 1. Aggregate Insurance

In [6]:
# Base directory
state_path = Path(state_directories[0])

print('Aggregation Insurance Path', state_path)

records = []  # Efficient list-based collection

# Loop over all JSON files recursively (State/Year/Quarter.json)
for json_file in state_path.glob('*/*/*.json'):
    state = json_file.parent.parent.name
    year = json_file.parent.name
    quarter = int(json_file.stem)

    try:
        # Faster than pd.read_json for nested JSONs
        with open(json_file, 'r') as f:
            data = json.load(f)

        # Extract transaction data safely
        insurances = data.get('data', {}).get('transactionData', [])
        for txn in insurances:
            records.append({
                'State': state,
                'Year': year,
                'Quarter': quarter,
                'Transaction_type': txn.get('name'),
                'Transaction_count': txn.get('paymentInstruments', [{}])[0].get('count'),
                'Transaction_amount': txn.get('paymentInstruments', [{}])[0].get('amount')
            })
    except (ValueError, KeyError, TypeError, FileNotFoundError, json.JSONDecodeError):
        continue

# Build the DataFrame once (very efficient)
agg_insur_df = pd.DataFrame(records)
agg_insur_df

Aggregation Insurance Path D:\Labmentix\phonpe-pulse-visualization\Pulse\data\aggregated\insurance\country\india\state


,State,Year,Quarter,Transaction_type,Transaction_count,Transaction_amount
0,Andaman And Nicobar Islands,2020,2,Insurance,6,1360.0
1,Andaman And Nicobar Islands,2020,3,Insurance,41,15380.0
2,Andaman And Nicobar Islands,2020,4,Insurance,124,157975.0
3,Andaman And Nicobar Islands,2021,1,Insurance,225,244266.0
4,Andaman And Nicobar Islands,2021,2,Insurance,137,181504.0
...,...,...,...,...,...,...
677,West Bengal,2023,4,Insurance,72712,100365562.0
678,West Bengal,2024,1,Insurance,79576,104987909.0
679,West Bengal,2024,2,Insurance,67048,89476633.0
680,West Bengal,2024,3,Insurance,77158,107451766.0


##### 2. Aggregate Transaction

In [7]:
# Base directory
state_path = Path(state_directories[1])

print('Aggregation Transaction Path', state_path)

records = []  # Efficient list-based collection

# Loop over all JSON files recursively (State/Year/Quarter.json)
for json_file in state_path.glob('*/*/*.json'):
    state = json_file.parent.parent.name
    year = json_file.parent.name
    quarter = int(json_file.stem)

    try:
        # Faster than pd.read_json for nested JSONs
        with open(json_file, 'r') as f:
            data = json.load(f)

        # Extract transaction data safely
        transactions = data.get('data', {}).get('transactionData', [])
        for txn in transactions:
            records.append({
                'State': state,
                'Year': year,
                'Quarter': quarter,
                'Transaction_type': txn.get('name'),
                'Transaction_count': txn.get('paymentInstruments', [{}])[0].get('count'),
                'Transaction_amount': txn.get('paymentInstruments', [{}])[0].get('amount')
            })
    except (ValueError, KeyError, TypeError, FileNotFoundError, json.JSONDecodeError):
        continue

# Build the DataFrame once (very efficient)
agg_trans_df = pd.DataFrame(records)
agg_trans_df

Aggregation Transaction Path D:\Labmentix\phonpe-pulse-visualization\Pulse\data\aggregated\transaction\country\india\state


,State,Year,Quarter,Transaction_type,Transaction_count,Transaction_amount
0,Andaman And Nicobar Islands,2018,1,Recharge & bill payments,4200,1.845307e+06
1,Andaman And Nicobar Islands,2018,1,Peer-to-peer payments,1871,1.213866e+07
2,Andaman And Nicobar Islands,2018,1,Merchant payments,298,4.525072e+05
3,Andaman And Nicobar Islands,2018,1,Financial Services,33,1.060142e+04
4,Andaman And Nicobar Islands,2018,1,Others,256,1.846899e+05
...,...,...,...,...,...,...
5029,West Bengal,2024,4,Merchant payments,655100809,3.892862e+11
5030,West Bengal,2024,4,Peer-to-peer payments,493217788,1.361927e+12
5031,West Bengal,2024,4,Recharge & bill payments,76043195,5.753406e+10
5032,West Bengal,2024,4,Financial Services,2352084,8.472965e+08


##### 3. Aggregate User

In [8]:
# Base path
state_path = Path(state_directories[2])

print('Aggregation Users Path', state_path)

records = []  # Instead of building a dict piece-by-piece

# Loop through all JSON files recursively
for json_file in state_path.glob('*/*/*.json'):
    state = json_file.parent.parent.name
    year = json_file.parent.name
    quarter = int(json_file.stem)

    try:
        # Load JSON more efficiently
        with open(json_file, 'r') as f:
            data = json.load(f)

        users = data.get('data', {}).get('usersByDevice', [])
        for u in users:
            records.append({
                'State': state,
                'Year': year,
                'Quarter': quarter,
                'Brand': u.get('brand'),
                'Transaction_count': u.get('count'),
                'Percentage': u.get('percentage')
            })
    except (ValueError, KeyError, TypeError, FileNotFoundError):
        continue

# Convert list of dicts to DataFrame at once (much faster)
agg_user_df = pd.DataFrame(records)
agg_user_df

Aggregation Users Path D:\Labmentix\phonpe-pulse-visualization\Pulse\data\aggregated\user\country\india\state


,State,Year,Quarter,Brand,Transaction_count,Percentage
0,Andaman And Nicobar Islands,2018,1,Xiaomi,1665,0.247033
1,Andaman And Nicobar Islands,2018,1,Samsung,1445,0.214392
2,Andaman And Nicobar Islands,2018,1,Vivo,982,0.145697
3,Andaman And Nicobar Islands,2018,1,Oppo,501,0.074332
4,Andaman And Nicobar Islands,2018,1,OnePlus,332,0.049258
...,...,...,...,...,...,...
6727,West Bengal,2022,1,Lenovo,330017,0.015056
6728,West Bengal,2022,1,Infinix,284678,0.012987
6729,West Bengal,2022,1,Asus,280347,0.012790
6730,West Bengal,2022,1,Apple,277752,0.012671


##### 4. Map Insurance

In [9]:
# Base directory
state_path = Path(state_directories[4])

records = []  # Store all rows efficiently here

# Loop through all JSON files recursively (State/Year/Quarter.json)
for json_file in state_path.glob('*/*/*.json'):
    state = json_file.parent.parent.name
    year = json_file.parent.name
    quarter = int(json_file.stem)

    try:
        with open(json_file, 'r') as f:
            data = json.load(f)

        hover_list = data.get('data', {}).get('hoverDataList', [])
        for item in hover_list:
            district = (
                item.get('name', '')
                .removesuffix(' district')
                .title()
                .replace(' And', ' and')
                .replace('andaman', 'Andaman')
            )

            metrics = item.get('metric', [{}])[0]
            records.append({
                'State': state,
                'Year': year,
                'Quarter': quarter,
                'District': district,
                'Transaction_count': metrics.get('count'),
                'Transaction_amount': metrics.get('amount')
            })
    except (ValueError, KeyError, TypeError, FileNotFoundError, json.JSONDecodeError):
        continue

# Create the final DataFrame
map_insur_df = pd.DataFrame(records)
map_insur_df

,State,Year,Quarter,District,Transaction_count,Transaction_amount
0,Andaman And Nicobar Islands,2020,2,South Andaman,3,795.0
1,Andaman And Nicobar Islands,2020,2,Nicobars,3,565.0
2,Andaman And Nicobar Islands,2020,3,North and Middle Andaman,1,281.0
3,Andaman And Nicobar Islands,2020,3,South Andaman,35,13651.0
4,Andaman And Nicobar Islands,2020,3,Nicobars,5,1448.0
...,...,...,...,...,...,...
13871,West Bengal,2024,4,Alipurduar,1023,1613143.0
13872,West Bengal,2024,4,Paschim Bardhaman,4945,7005851.0
13873,West Bengal,2024,4,Nadia,3807,5031294.0
13874,West Bengal,2024,4,Birbhum,1818,2423290.0


##### 5. Map Transaction

In [10]:
# Base directory
state_path = Path(state_directories[5])

records = []  # Store all rows efficiently here

# Loop through all JSON files recursively (State/Year/Quarter.json)
for json_file in state_path.glob('*/*/*.json'):
    state = json_file.parent.parent.name
    year = json_file.parent.name
    quarter = int(json_file.stem)

    try:
        with open(json_file, 'r') as f:
            data = json.load(f)

        hover_list = data.get('data', {}).get('hoverDataList', [])
        for item in hover_list:
            district = (
                item.get('name', '')
                .removesuffix(' district')
                .title()
                .replace(' And', ' and')
                .replace('andaman', 'Andaman')
            )

            metrics = item.get('metric', [{}])[0]
            records.append({
                'State': state,
                'Year': year,
                'Quarter': quarter,
                'District': district,
                'Transaction_count': metrics.get('count'),
                'Transaction_amount': metrics.get('amount')
            })
    except (ValueError, KeyError, TypeError, FileNotFoundError, json.JSONDecodeError):
        continue

# Create the final DataFrame
map_trans_df = pd.DataFrame(records)
map_trans_df

,State,Year,Quarter,District,Transaction_count,Transaction_amount
0,Andaman And Nicobar Islands,2018,1,North and Middle Andaman,442,9.316631e+05
1,Andaman And Nicobar Islands,2018,1,South Andaman,5688,1.256025e+07
2,Andaman And Nicobar Islands,2018,1,Nicobars,528,1.139849e+06
3,Andaman And Nicobar Islands,2018,2,North and Middle Andaman,825,1.317863e+06
4,Andaman And Nicobar Islands,2018,2,South Andaman,9395,2.394824e+07
...,...,...,...,...,...,...
20599,West Bengal,2024,4,Alipurduar,15875637,2.099251e+10
20600,West Bengal,2024,4,Paschim Bardhaman,56616799,6.968735e+10
20601,West Bengal,2024,4,Nadia,65274337,1.079320e+11
20602,West Bengal,2024,4,Birbhum,36905213,5.778701e+10


##### 6. Map User

In [11]:
# Base directory
state_path = Path(state_directories[6])

records = []  # Efficient list-based collection

# Traverse all JSON files (State/Year/Quarter.json)
for json_file in state_path.glob('*/*/*.json'):
    state = json_file.parent.parent.name
    year = json_file.parent.name
    quarter = int(json_file.stem)

    try:
        with open(json_file, 'r') as f:
            data = json.load(f)

        hover_data = data.get('data', {}).get('hoverData', {})
        for district, user_data in hover_data.items():
            district_name = (
                district.removesuffix(' district')
                .title()
                .replace(' And', ' and')
                .replace('andaman', 'Andaman')
            )

            records.append({
                'State': state,
                'Year': year,
                'Quarter': quarter,
                'District': district_name,
                'Registered_users': user_data.get('registeredUsers'),
                'App_opens': user_data.get('appOpens')
            })
    except (ValueError, KeyError, TypeError, FileNotFoundError, json.JSONDecodeError):
        continue

# Build DataFrame once (faster)
map_user_df = pd.DataFrame(records)
map_user_df

,State,Year,Quarter,District,Registered_users,App_opens
0,Andaman And Nicobar Islands,2018,1,North and Middle Andaman,632,0
1,Andaman And Nicobar Islands,2018,1,South Andaman,5846,0
2,Andaman And Nicobar Islands,2018,1,Nicobars,262,0
3,Andaman And Nicobar Islands,2018,2,North and Middle Andaman,911,0
4,Andaman And Nicobar Islands,2018,2,South Andaman,8143,0
...,...,...,...,...,...,...
20603,West Bengal,2024,4,Alipurduar,475688,31842355
20604,West Bengal,2024,4,Paschim Bardhaman,1468252,80543469
20605,West Bengal,2024,4,Nadia,1861738,98740305
20606,West Bengal,2024,4,Birbhum,1114220,73465525


##### 6. Top Insurances District-wise

In [12]:
# Base directory
state_path = Path(state_directories[7])

records = []  # Collect all rows efficiently

# Loop over all JSON files recursively (State/Year/Quarter.json)
for json_file in state_path.glob('*/*/*.json'):
    state = json_file.parent.parent.name
    year = json_file.parent.name
    quarter = int(json_file.stem)

    try:
        with open(json_file, 'r') as f:
            data = json.load(f)

        districts = data.get('data', {}).get('districts', [])
        for district_data in districts:
            name = (
                district_data.get('entityName', '')
                .title()
                .replace(' And', ' and')
                .replace('andaman', 'Andaman')
            )

            metric = district_data.get('metric', {})
            records.append({
                'State': state,
                'Year': year,
                'Quarter': quarter,
                'District': name,
                'Transaction_count': metric.get('count'),
                'Transaction_amount': metric.get('amount')
            })

    except (ValueError, KeyError, TypeError, FileNotFoundError, json.JSONDecodeError):
        continue

# Create DataFrame once (efficient)
top_insur_dist_df = pd.DataFrame(records)
top_insur_dist_df

,State,Year,Quarter,District,Transaction_count,Transaction_amount
0,Andaman And Nicobar Islands,2020,2,Nicobars,3,565.0
1,Andaman And Nicobar Islands,2020,2,South Andaman,3,795.0
2,Andaman And Nicobar Islands,2020,3,South Andaman,35,13651.0
3,Andaman And Nicobar Islands,2020,3,Nicobars,5,1448.0
4,Andaman And Nicobar Islands,2020,3,North and Middle Andaman,1,281.0
...,...,...,...,...,...,...
5603,West Bengal,2024,4,Paschim Bardhaman,4945,7005851.0
5604,West Bengal,2024,4,Paschim Medinipur,4155,5405054.0
5605,West Bengal,2024,4,Malda,4042,4902098.0
5606,West Bengal,2024,4,Nadia,3807,5031294.0


##### 7. Top Insurance Pincode-wise

In [13]:
# Base directory
state_path = Path(state_directories[8])

records = []  # Efficient collection of rows

# Loop through all JSON files recursively (State/Year/Quarter.json)
for json_file in state_path.glob('*/*/*.json'):
    state = json_file.parent.parent.name
    year = json_file.parent.name
    quarter = int(json_file.stem)

    try:
        with open(json_file, 'r') as f:
            data = json.load(f)

        pincodes = data.get('data', {}).get('pincodes', [])
        for pin_data in pincodes:
            records.append({
                'State': state,
                'Year': year,
                'Quarter': quarter,
                'Pincode': pin_data.get('entityName'),
                'Transaction_count': pin_data.get('metric', {}).get('count'),
                'Transaction_amount': pin_data.get('metric', {}).get('amount')
            })

    except (ValueError, KeyError, TypeError, FileNotFoundError, json.JSONDecodeError):
        continue

# Build DataFrame once (much faster)
top_insur_pin_df = pd.DataFrame(records)
top_insur_pin_df

,State,Year,Quarter,Pincode,Transaction_count,Transaction_amount
0,Andaman And Nicobar Islands,2018,1,744101,1622,2.769298e+06
1,Andaman And Nicobar Islands,2018,1,744103,1223,2.238042e+06
2,Andaman And Nicobar Islands,2018,1,744102,969,3.519060e+06
3,Andaman And Nicobar Islands,2018,1,744105,685,1.298561e+06
4,Andaman And Nicobar Islands,2018,1,744104,340,1.039715e+06
...,...,...,...,...,...,...
9994,West Bengal,2024,4,711101,6753348,1.049483e+10
9995,West Bengal,2024,4,700059,6662715,8.009476e+09
9996,West Bengal,2024,4,700039,6575693,9.018748e+09
9997,West Bengal,2024,4,734001,6432082,8.996529e+09


##### 8. Top Transaction District-wise

In [14]:
# Base directory
state_path = Path(state_directories[8])

records = []  # Collect all rows efficiently

# Loop over all JSON files recursively (State/Year/Quarter.json)
for json_file in state_path.glob('*/*/*.json'):
    state = json_file.parent.parent.name
    year = json_file.parent.name
    quarter = int(json_file.stem)

    try:
        with open(json_file, 'r') as f:
            data = json.load(f)

        districts = data.get('data', {}).get('districts', [])
        for district_data in districts:
            name = (
                district_data.get('entityName', '')
                .title()
                .replace(' And', ' and')
                .replace('andaman', 'Andaman')
            )

            metric = district_data.get('metric', {})
            records.append({
                'State': state,
                'Year': year,
                'Quarter': quarter,
                'District': name,
                'Transaction_count': metric.get('count'),
                'Transaction_amount': metric.get('amount')
            })

    except (ValueError, KeyError, TypeError, FileNotFoundError, json.JSONDecodeError):
        continue

# Create DataFrame once (efficient)
top_trans_dist_df = pd.DataFrame(records)
top_trans_dist_df

,State,Year,Quarter,District,Transaction_count,Transaction_amount
0,Andaman And Nicobar Islands,2018,1,South Andaman,5688,1.256025e+07
1,Andaman And Nicobar Islands,2018,1,Nicobars,528,1.139849e+06
2,Andaman And Nicobar Islands,2018,1,North and Middle Andaman,442,9.316631e+05
3,Andaman And Nicobar Islands,2018,2,South Andaman,9395,2.394824e+07
4,Andaman And Nicobar Islands,2018,2,Nicobars,1120,3.072437e+06
...,...,...,...,...,...,...
8291,West Bengal,2024,4,Howrah,67496284,1.025764e+11
8292,West Bengal,2024,4,Nadia,65274337,1.079320e+11
8293,West Bengal,2024,4,Malda,63424613,1.024212e+11
8294,West Bengal,2024,4,Hooghly,59676576,9.071383e+10


##### 9. Top Transaction Pincode-wise 

In [15]:
# Base directory
state_path = Path(state_directories[8])

records = []  # Efficient collection of rows

# Loop through all JSON files recursively (State/Year/Quarter.json)
for json_file in state_path.glob('*/*/*.json'):
    state = json_file.parent.parent.name
    year = json_file.parent.name
    quarter = int(json_file.stem)

    try:
        with open(json_file, 'r') as f:
            data = json.load(f)

        pincodes = data.get('data', {}).get('pincodes', [])
        for pin_data in pincodes:
            records.append({
                'State': state,
                'Year': year,
                'Quarter': quarter,
                'Pincode': pin_data.get('entityName'),
                'Transaction_count': pin_data.get('metric', {}).get('count'),
                'Transaction_amount': pin_data.get('metric', {}).get('amount')
            })

    except (ValueError, KeyError, TypeError, FileNotFoundError, json.JSONDecodeError):
        continue

# Build DataFrame once (much faster)
top_trans_pin_df = pd.DataFrame(records)
top_trans_pin_df

,State,Year,Quarter,Pincode,Transaction_count,Transaction_amount
0,Andaman And Nicobar Islands,2018,1,744101,1622,2.769298e+06
1,Andaman And Nicobar Islands,2018,1,744103,1223,2.238042e+06
2,Andaman And Nicobar Islands,2018,1,744102,969,3.519060e+06
3,Andaman And Nicobar Islands,2018,1,744105,685,1.298561e+06
4,Andaman And Nicobar Islands,2018,1,744104,340,1.039715e+06
...,...,...,...,...,...,...
9994,West Bengal,2024,4,711101,6753348,1.049483e+10
9995,West Bengal,2024,4,700059,6662715,8.009476e+09
9996,West Bengal,2024,4,700039,6575693,9.018748e+09
9997,West Bengal,2024,4,734001,6432082,8.996529e+09


##### 10. Top User District-wise

In [16]:
# Base directory for this dataset
state_path = Path(state_directories[9])

records = []  # Collect all rows efficiently

# Loop through all JSON files recursively (State/Year/Quarter.json)
for json_file in state_path.glob('*/*/*.json'):
    state = json_file.parent.parent.name
    year = json_file.parent.name
    quarter = int(json_file.stem)

    try:
        with open(json_file, 'r') as f:
            data = json.load(f)

        districts = data.get('data', {}).get('districts', [])
        for district_data in districts:
            name = (
                district_data.get('name', '')
                .title()
                .replace(' And', ' and')
                .replace('andaman', 'Andaman')
            )

            records.append({
                'State': state,
                'Year': year,
                'Quarter': quarter,
                'District': name,
                'Registered_users': district_data.get('registeredUsers')
            })

    except (ValueError, KeyError, TypeError, FileNotFoundError, json.JSONDecodeError):
        continue

# Convert once (efficient)
top_user_dist_df = pd.DataFrame(records)
top_user_dist_df

,State,Year,Quarter,District,Registered_users
0,Andaman And Nicobar Islands,2018,1,South Andaman,5846
1,Andaman And Nicobar Islands,2018,1,North and Middle Andaman,632
2,Andaman And Nicobar Islands,2018,1,Nicobars,262
3,Andaman And Nicobar Islands,2018,2,South Andaman,8143
4,Andaman And Nicobar Islands,2018,2,North and Middle Andaman,911
...,...,...,...,...,...
8291,West Bengal,2024,4,Hooghly,1957356
8292,West Bengal,2024,4,Nadia,1861738
8293,West Bengal,2024,4,Purba Medinipur,1717074
8294,West Bengal,2024,4,Paschim Medinipur,1522845


##### 11. Top User Pincode-wise

In [17]:
# Base path
state_path = Path(state_directories[9])

records = []  # Efficient list for all extracted rows

# Iterate through all JSON files recursively: State/Year/Quarter.json
for json_file in state_path.glob('*/*/*.json'):
    state = json_file.parent.parent.name
    year = json_file.parent.name
    quarter = int(json_file.stem)

    try:
        with open(json_file, 'r') as f:
            data = json.load(f)

        pincodes = data.get('data', {}).get('pincodes', [])
        for pin_data in pincodes:
            records.append({
                'State': state,
                'Year': year,
                'Quarter': quarter,
                'Pincode': pin_data.get('name'),
                'Registered_users': pin_data.get('registeredUsers')
            })

    except (ValueError, KeyError, TypeError, FileNotFoundError, json.JSONDecodeError):
        continue

# Build DataFrame once (much faster than appending in loop)
top_user_pin_df = pd.DataFrame(records)

### List of dataframes

In [18]:
df_list = [df for df in globals() if isinstance(globals()[df], pd.core.frame.DataFrame) and df.endswith('_df')]
# df_list = [
#     name for name, df in globals().items()
#     if isinstance(df, pd.DataFrame) and name.endswith('_df') and not df.empty
# ]
df_list

['agg_insur_df',
 'agg_trans_df',
 'agg_user_df',
 'map_insur_df',
 'map_trans_df',
 'map_user_df',
 'top_insur_dist_df',
 'top_insur_pin_df',
 'top_trans_dist_df',
 'top_trans_pin_df',
 'top_user_dist_df',
 'top_user_pin_df']

Add Suffix to State to maintian consistency

In [19]:
# As I noticed few district name is mismatched between dfs loaded from pulse and lat_long_df, doing this.

def add_suffix_to_districts(df):
    if 'District' in df.columns and 'State' in df.columns:
        delhi_df = df[df['State'] == 'Delhi']
        
        districts_to_suffix = [d for d in delhi_df['District'].unique() if d != 'Shahdara']
        
        df.loc[(df['State'] == 'Delhi') & (df['District'].isin(districts_to_suffix)), 'District'] = df.loc[(df['State'] == 'Delhi') & (df['District'].isin(districts_to_suffix)), 'District'].apply(lambda x: x + ' Delhi' if 'Delhi' not in x else x)

    return df

for df_name in df_list:
    df = globals()[df_name]
    add_suffix_to_districts(df)

Adding Latitude and Longitude columns

In [ ]:
lat_long_df = pd.read_csv(r"D:\Labmentix\phonpe-pulse-visualization\utils\dist_lat_long.csv")

for df_name in df_list:
    df = globals()[df_name]
    if 'District' in df.columns:
        df = pd.merge(df, lat_long_df, on=['State', 'District'], how='left')
        globals()[df_name] = df

Adding Region column to all dataframes

In [21]:
def add_region_column(df):
    state_groups = {
        'Northern Region': ['Jammu and Kashmir', 'Himachal Pradesh', 'Punjab', 'Chandigarh', 'Uttarakhand', 'Ladakh', 'Delhi', 'Haryana'],
        'Central Region': ['Uttar Pradesh', 'Madhya Pradesh', 'Chhattisgarh'],
        'Western Region': ['Rajasthan', 'Gujarat', 'Dadra And Nagar Haveli And Daman and Diu', 'Maharashtra'],
        'Eastern Region': ['Bihar', 'Jharkhand', 'Odisha', 'West Bengal', 'Sikkim'],
        'Southern Region': ['Andhra Pradesh', 'Telangana', 'Karnataka', 'Kerala', 'Tamil Nadu', 'Puducherry', 'Goa', 'Lakshadweep', 'Andaman And Nicobar Islands'],
        'North-Eastern Region': ['Assam', 'Meghalaya', 'Manipur', 'Nagaland', 'Tripura', 'Arunachal Pradesh', 'Mizoram']
    }

    if 'State' not in df.columns:
        print("⚠️ Skipping DataFrame: 'State' column not found", df.columns)
        return df  # Return unchanged DataFrame
    
    df['Region'] = df['State'].map({state: region for region, states in state_groups.items() for state in states})
    return df

In [22]:
for df_name in df_list:
    df = globals()[df_name]
    print(df)
    add_region_column(df)

                           State  Year  Quarter Transaction_type  \
0    Andaman And Nicobar Islands  2020        2        Insurance   
1    Andaman And Nicobar Islands  2020        3        Insurance   
2    Andaman And Nicobar Islands  2020        4        Insurance   
3    Andaman And Nicobar Islands  2021        1        Insurance   
4    Andaman And Nicobar Islands  2021        2        Insurance   
..                           ...   ...      ...              ...   
677                  West Bengal  2023        4        Insurance   
678                  West Bengal  2024        1        Insurance   
679                  West Bengal  2024        2        Insurance   
680                  West Bengal  2024        3        Insurance   
681                  West Bengal  2024        4        Insurance   

     Transaction_count  Transaction_amount  
0                    6              1360.0  
1                   41             15380.0  
2                  124            157975.0  
3  

### Columnwise null-count and duplicated_rows-count

In [23]:
for df_name in df_list:
    df = globals()[df_name]
    print(f"{df_name}:")
    print(f"Null count: \n{df.isnull().sum()}")
    print(f"Duplicated rows count: \n{df.duplicated().sum()}")
    print(df.shape)
    print("\n", 25 * "_", "\n")

agg_insur_df:
Null count: 
State                  0
Year                   0
Quarter                0
Transaction_type       0
Transaction_count      0
Transaction_amount     0
Region                38
dtype: int64
Duplicated rows count: 
0
(682, 7)

 _________________________ 

agg_trans_df:
Null count: 
State                   0
Year                    0
Quarter                 0
Transaction_type        0
Transaction_count       0
Transaction_amount      0
Region                280
dtype: int64
Duplicated rows count: 
0
(5034, 7)

 _________________________ 

agg_user_df:
Null count: 
State                  0
Year                   0
Quarter                0
Brand                  0
Transaction_count      0
Percentage             0
Region               374
dtype: int64
Duplicated rows count: 
0
(6732, 7)

 _________________________ 

map_insur_df:
Null count: 
State                   0
Year                    0
Quarter                 0
District                0
Transaction_count    

### Understanding the dataframes

In [24]:
print('DATAFRAME INFO:\n')

for df_name in df_list:
    df = globals()[df_name]
    print(df_name + ':\n')
    df.info()
    print("\n", 45 * "_", "\n")

DATAFRAME INFO:

agg_insur_df:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 682 entries, 0 to 681
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   State               682 non-null    object 
 1   Year                682 non-null    object 
 2   Quarter             682 non-null    int64  
 3   Transaction_type    682 non-null    object 
 4   Transaction_count   682 non-null    int64  
 5   Transaction_amount  682 non-null    float64
 6   Region              644 non-null    object 
dtypes: float64(1), int64(2), object(4)
memory usage: 37.4+ KB

 _____________________________________________ 

agg_trans_df:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5034 entries, 0 to 5033
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   State               5034 non-null   object 
 1   Year                5034 non-null   o

Dropping rows with null values

In [25]:
# 'top_trans_pin_df' seems to have two null values and they are not of significant proportion so dropping them;

top_trans_pin_df.dropna(axis = 'index', inplace = True)
top_trans_pin_df.isnull().sum()

State                 0
Year                  0
Quarter               0
Pincode               0
Transaction_count     0
Transaction_amount    0
Region                0
dtype: int64

Changing datatype of year column to int across all dataframes

In [26]:
# Year column in all the dataframes seems to be of object dtype so changing it to int object so as to push into MySQL as year;

for df_name in df_list:
    df = globals()[df_name]
    print(df)
    # df['Year'] = df['Year'].astype('int')

                           State  Year  Quarter Transaction_type  \
0    Andaman And Nicobar Islands  2020        2        Insurance   
1    Andaman And Nicobar Islands  2020        3        Insurance   
2    Andaman And Nicobar Islands  2020        4        Insurance   
3    Andaman And Nicobar Islands  2021        1        Insurance   
4    Andaman And Nicobar Islands  2021        2        Insurance   
..                           ...   ...      ...              ...   
677                  West Bengal  2023        4        Insurance   
678                  West Bengal  2024        1        Insurance   
679                  West Bengal  2024        2        Insurance   
680                  West Bengal  2024        3        Insurance   
681                  West Bengal  2024        4        Insurance   

     Transaction_count  Transaction_amount           Region  
0                    6              1360.0  Southern Region  
1                   41             15380.0  Southern Region

### Outlier across all dataframes

In [27]:
# Everything seems to be alright as far as dtypes and nullvalues are concerned so checking for outliers
# Function to check for outliers

def count_outliers(df):
    outliers = {}
    for col in df.select_dtypes(include=[np.number]).columns:
        if col in ['Transaction_count', 'Transaction_amount']:
            q1 = df[col].quantile(0.25)
            q3 = df[col].quantile(0.75)
            iqr = q3 - q1
            upper_bound = q3 + (1.5 * iqr)
            lower_bound = q1 - (1.5 * iqr)
            outliers[col] = len(df[(df[col] > upper_bound) | (df[col] < lower_bound)])
        else:
            continue
    return outliers

print('OUTLIER COUNT ACROSS DATAFRAMES:\n')

for df_name in df_list:
    df = globals()[df_name]
    outliers = count_outliers(df)
    if len(outliers) == 0:
        pass
    else:
        print(df_name, ":\n\n", outliers, "\n")
        print("\n", 55 * "_", "\n")

OUTLIER COUNT ACROSS DATAFRAMES:

agg_insur_df :

 {'Transaction_count': 49, 'Transaction_amount': 69} 


 _______________________________________________________ 

agg_trans_df :

 {'Transaction_count': 936, 'Transaction_amount': 955} 


 _______________________________________________________ 

agg_user_df :

 {'Transaction_count': 893} 


 _______________________________________________________ 

map_insur_df :

 {'Transaction_count': 1534, 'Transaction_amount': 1502} 


 _______________________________________________________ 

map_trans_df :

 {'Transaction_count': 2643, 'Transaction_amount': 2469} 


 _______________________________________________________ 

top_insur_dist_df :

 {'Transaction_count': 484, 'Transaction_amount': 508} 


 _______________________________________________________ 

top_insur_pin_df :

 {'Transaction_count': 1412, 'Transaction_amount': 1366} 


 _______________________________________________________ 

top_trans_dist_df :

 {'Transaction_count': 1144, 

### Unique value count across all dataframes

In [28]:
# Function to check for unique value counts and print if count less than 10;

def unique_value_count(df, exclude_cols=[]):
    for col in df.columns:
        if col in exclude_cols:
            continue
        unique_vals = df[col].nunique()
        print(f"{col}: {unique_vals} unique values")
        if unique_vals < 10:
            print(df[col].unique())

print('UNIQUE VALUE COUNT ACROSS DATAFRAMES; \n')

for df_name in df_list:
    df = globals()[df_name]
    print(df_name, ":\n")
    unique_value_count(df, exclude_cols = ['State', 'Year', 'Quarter', 'Percentage'])
    print("\n", 55 * "_", "\n")

UNIQUE VALUE COUNT ACROSS DATAFRAMES; 

agg_insur_df :

Transaction_type: 1 unique values
['Insurance']
Transaction_count: 654 unique values
Transaction_amount: 682 unique values
Region: 6 unique values
['Southern Region' 'North-Eastern Region' 'Eastern Region'
 'Northern Region' 'Central Region' nan 'Western Region']

 _______________________________________________________ 

agg_trans_df :

Transaction_type: 5 unique values
['Recharge & bill payments' 'Peer-to-peer payments' 'Merchant payments'
 'Financial Services' 'Others']
Transaction_count: 4966 unique values
Transaction_amount: 5034 unique values
Region: 6 unique values
['Southern Region' 'North-Eastern Region' 'Eastern Region'
 'Northern Region' 'Central Region' nan 'Western Region']

 _______________________________________________________ 

agg_user_df :

Brand: 20 unique values
Transaction_count: 6501 unique values
Region: 6 unique values
['Southern Region' 'North-Eastern Region' 'Eastern Region'
 'Northern Region' 'Central 

### Creating CSV files out of the refined dataframes for ease in manipulation

In [ ]:
def save_dfs_as_csv(df_list):
    subfolder = r"D:\Labmentix\phonpe-pulse-visualization\app\out"
    if not os.path.exists(subfolder):
        os.makedirs(subfolder)
        
    for df_name in df_list:
        df = globals()[df_name]
        file_path = os.path.join(subfolder, df_name.replace('_df', '') + '.csv')
        df.to_csv(file_path, index=False)

# Calling function to execute

save_dfs_as_csv(df_list)

### Pushing data to Postgres Database

Establishing connection and creating cursor

In [30]:
conn = psycopg2.connect(
  host = "localhost",
  user = "postgres",
  password = "password"
)

conn.autocommit = True

cursor = conn.cursor()

Database Creation

In [31]:
# Drop the database if it exists
cursor.execute("DROP DATABASE IF EXISTS phonepe_pulse;")

cursor.execute("CREATE DATABASE phonepe_pulse")

conn = psycopg2.connect(
  host = "localhost",
  user = "postgres",
  database = "phonepe_pulse",
  password = "password"
)

conn.autocommit = True

cursor = conn.cursor()

Creating Tables

In [32]:
# Query for create table aggegation insurance
cursor.execute('''          
CREATE TABLE IF NOT EXISTS agg_insur (
    State VARCHAR(255),
    Year INTEGER,
    Quarter INTEGER,
    Transaction_type VARCHAR(255),
    Transaction_count INTEGER,
    Transaction_amount FLOAT,
    Region VARCHAR(255),
    PRIMARY KEY (State, Year, Quarter, Transaction_type, Region)
)
''')

# Query for create table aggegation transaction
cursor.execute('''          
CREATE TABLE IF NOT EXISTS agg_trans (
    State VARCHAR(255),
    Year INTEGER,
    Quarter INTEGER,
    Transaction_type VARCHAR(255),
    Transaction_count INTEGER,
    Transaction_amount FLOAT,
    Region VARCHAR(255),
    PRIMARY KEY (State, Year, Quarter, Transaction_type, Region)
)
''')

# Query for create table aggegation user
cursor.execute('''
CREATE TABLE IF NOT EXISTS agg_user (
    State VARCHAR(255),
    Year INTEGER,
    Quarter INTEGER,
    Brand VARCHAR(255),
    Transaction_count INTEGER,
    Percentage FLOAT,
    Region VARCHAR(255),
    PRIMARY KEY (State, Year, Quarter, Brand, Region)
)
''')

# Query for create table map insurance
cursor.execute('''
CREATE TABLE IF NOT EXISTS map_insur (
    State VARCHAR(255),
    Year INTEGER,
    Quarter INTEGER,
    District VARCHAR(255),
    Transaction_count INTEGER,
    Transaction_amount FLOAT,
    Latitude FLOAT,
    Longitude FLOAT,
    Region VARCHAR(255),
    PRIMARY KEY (State, Year, Quarter, District, Region)
)
''')

# Query for create table map transaction
cursor.execute('''
CREATE TABLE IF NOT EXISTS map_trans (
    State VARCHAR(255),
    Year INTEGER,
    Quarter INTEGER,
    District VARCHAR(255),
    Transaction_count INTEGER,
    Transaction_amount FLOAT,
    Latitude FLOAT,
    Longitude FLOAT,
    Region VARCHAR(255),
    PRIMARY KEY (State, Year, Quarter, District, Region)
)
''')

# Query for create table map user
cursor.execute('''
CREATE TABLE IF NOT EXISTS map_user (
    State VARCHAR(255),
    Year INTEGER,
    Quarter INTEGER,
    District VARCHAR(255),
    Registered_users INTEGER,
    App_opens INTEGER,
    Latitude FLOAT,
    Longitude FLOAT,
    Region VARCHAR(255),
    PRIMARY KEY (State, Year, Quarter, District, Region)
)
''')

# Query for create table top insurance district wise
cursor.execute('''
CREATE TABLE IF NOT EXISTS top_insur_dist (
    State VARCHAR(255),
    Year INTEGER,
    Quarter INTEGER,
    District VARCHAR(255),
    Transaction_count INTEGER,
    Transaction_amount FLOAT,
    Latitude FLOAT,
    Longitude FLOAT,
    Region VARCHAR(255),
    PRIMARY KEY (State, Year, Quarter, District, Region)
)
''')

# Query for create table top insurance pincode wise
cursor.execute('''
CREATE TABLE IF NOT EXISTS top_insur_pin (
    State VARCHAR(255),
    Year INTEGER,
    Quarter INTEGER,
    Pincode VARCHAR(255),
    Transaction_count INTEGER,
    Transaction_amount FLOAT,
    Region VARCHAR(255),
    PRIMARY KEY (State, Year, Quarter, Pincode, Region)
)
''')

# Query for create table top transaction district wise
cursor.execute('''
CREATE TABLE IF NOT EXISTS top_trans_dist (
    State VARCHAR(255),
    Year INTEGER,
    Quarter INTEGER,
    District VARCHAR(255),
    Transaction_count INTEGER,
    Transaction_amount FLOAT,
    Latitude FLOAT,
    Longitude FLOAT,
    Region VARCHAR(255),
    PRIMARY KEY (State, Year, Quarter, District, Region)
)
''')

# Query for create table top transaction pincode wise
cursor.execute('''
CREATE TABLE IF NOT EXISTS top_trans_pin (
    State VARCHAR(255),
    Year INTEGER,
    Quarter INTEGER,
    Pincode VARCHAR(255),
    Transaction_count INTEGER,
    Transaction_amount FLOAT,
    Region VARCHAR(255),
    PRIMARY KEY (State, Year, Quarter, Pincode, Region)
)
''')

# Query for create table top user district wise
cursor.execute('''
CREATE TABLE IF NOT EXISTS top_user_dist (
    State VARCHAR(255),
    Year INTEGER,
    Quarter INTEGER,
    District VARCHAR(255),
    Registered_users INTEGER,
    Latitude FLOAT,
    Longitude FLOAT,
    Region VARCHAR(255),
    PRIMARY KEY (State, Year, Quarter, District, Region)
)
''')

# Query for create table top user pincode wise
cursor.execute('''
CREATE TABLE IF NOT EXISTS top_user_pin (
    State VARCHAR(255),
    Year INTEGER,
    Quarter INTEGER,
    Pincode VARCHAR(255),
    Registered_users INTEGER,
    Region VARCHAR(255),
    PRIMARY KEY (State, Year, Quarter, Pincode, Region)
)
''')


#### Pushing data into Database

In [33]:
# def push_data_into_postgres(conn, cursor, dfs, table_columns):
#     for table_name in dfs.keys():
#         df = dfs[table_name]
#         columns = table_columns[table_name]
#         placeholders = ', '.join(['%s'] * len(columns))
#         query = f"INSERT INTO {table_name} ({', '.join(columns)}) VALUES ({placeholders})"
#         for _, row in df.iterrows():
#             data = tuple(row[column] for column in columns)
#             cursor.execute(query, data)
#         conn.commit()
#     print("Data successfully pushed into Postgres tables")

def push_data_into_postgres(conn, cursor, dfs, table_columns):
    for table_name, df in dfs.items():
        columns = table_columns[table_name]
        placeholders = ', '.join(['%s'] * len(columns))
        columns_str = ', '.join(columns)
        
        # Build ON CONFLICT clause based on primary key
        conflict_columns_query = f"""
        SELECT a.attname
        FROM pg_index i
        JOIN pg_attribute a ON a.attrelid = i.indrelid AND a.attnum = ANY(i.indkey)
        WHERE i.indrelid = '{table_name}'::regclass AND i.indisprimary;
        """
        cursor.execute(conflict_columns_query)
        primary_keys = [row[0] for row in cursor.fetchall()]
        
        if primary_keys:
            conflict_clause = f"ON CONFLICT ({', '.join(primary_keys)}) DO NOTHING"
        else:
            conflict_clause = ""
        
        query = f"INSERT INTO {table_name} ({columns_str}) VALUES ({placeholders}) {conflict_clause}"
        
        for _, row in df.iterrows():
            data = tuple(row[col] for col in columns)
            try:
                cursor.execute(query, data)
            except Exception as e:
                print(f"Skipping row due to error: {e}")
        
        conn.commit()
    print("Data successfully pushed into Postgres tables (duplicates skipped)")


In [34]:
# Mapping my_sql tables to pandas dataframes that we have created earlier

dfs = {
    'agg_insur': agg_insur_df,
    'agg_trans': agg_trans_df,
    'agg_user': agg_user_df,
    'map_insur': map_insur_df,
    'map_trans': map_trans_df,
    'map_user': map_user_df,
    'top_insur_dist': top_insur_dist_df,
    'top_insur_pin': top_insur_pin_df,
    'top_trans_dist': top_trans_dist_df,
    'top_trans_pin': top_trans_pin_df,
    'top_user_dist': top_user_dist_df,
    'top_user_pin': top_user_pin_df
}

# Mapping table name to associated columns for each table

table_columns = {
    'agg_insur': list(agg_insur_df.columns),
    'agg_trans': list(agg_trans_df.columns),
    'agg_user': list(agg_user_df.columns),
    'map_insur': list(map_insur_df.columns),
    'map_trans': list(map_trans_df.columns),
    'map_user': list(map_user_df.columns),
    'top_insur_dist': list(top_insur_dist_df.columns),
    'top_insur_pin': list(top_insur_pin_df.columns),
    'top_trans_dist': list(top_trans_dist_df.columns),
    'top_trans_pin': list(top_trans_pin_df.columns),
    'top_user_dist': list(top_user_dist_df.columns),
    'top_user_pin': list(top_user_pin_df.columns)
}

In [35]:
push_data_into_postgres(conn, cursor, dfs, table_columns)

Skipping row due to error: integer out of range

Skipping row due to error: integer out of range

Skipping row due to error: null value in column "pincode" of relation "top_insur_pin" violates not-null constraint
DETAIL:  Failing row contains (Ladakh, 2019, 4, null, 2014, 10098656.16799601, Northern Region).

Skipping row due to error: null value in column "pincode" of relation "top_insur_pin" violates not-null constraint
DETAIL:  Failing row contains (Ladakh, 2020, 4, null, 13717, 36711603.92132313, Northern Region).

Data successfully pushed into Postgres tables (duplicates skipped)


#### Verifying shape of tables and dataframes are equal or not

In [36]:
# Get list of tables in database

cursor.execute("""
    SELECT table_name
    FROM information_schema.tables
    WHERE table_schema = 'public';
""")
tables = cursor.fetchall()
# print([table[0] for table in tables])


# Loop through tables and get count of rows and columns in MySQL

for table in tables:
    table_name = table[0]
    cursor.execute(f"SELECT COUNT(*) FROM {table_name}")
    row_count = cursor.fetchone()[0]
    cursor.execute(f"SELECT COUNT(*) FROM information_schema.columns WHERE table_name='{table_name}'")
    column_count = cursor.fetchone()[0]

    # Check if shape of DataFrame matches count of rows and columns in table

    df = dfs[table_name]
    if df.shape == (row_count, column_count):
        print(f"{table_name} table has {row_count} rows and {column_count} columns and shape matches DataFrame.")
    else:
        print(f"{table_name} table has {row_count} rows and {column_count} columns but shape does not match DataFrame.")


agg_insur table has 682 rows and 7 columns and shape matches DataFrame.
agg_trans table has 5032 rows and 7 columns but shape does not match DataFrame.
agg_user table has 6732 rows and 7 columns and shape matches DataFrame.
map_insur table has 13875 rows and 9 columns but shape does not match DataFrame.
map_trans table has 20604 rows and 9 columns and shape matches DataFrame.
map_user table has 20608 rows and 9 columns and shape matches DataFrame.
top_insur_dist table has 5608 rows and 9 columns and shape matches DataFrame.
top_insur_pin table has 9997 rows and 7 columns but shape does not match DataFrame.
top_trans_dist table has 8296 rows and 9 columns and shape matches DataFrame.
top_trans_pin table has 9463 rows and 7 columns and shape matches DataFrame.
top_user_dist table has 8296 rows and 8 columns and shape matches DataFrame.
top_user_pin table has 10000 rows and 6 columns and shape matches DataFrame.


#### Close Database Connection

In [37]:
cursor.close()
conn.close()